In [1]:
import cv2 as cv
import numpy as np
import math 
from skimage.feature import hog
import matplotlib.pyplot as plt
import os
import json
import pandas as pd
import csv
from sklearn.svm import SVC
from skimage.feature import graycoprops, graycomatrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
json_file = r"C:\Users\mdrsp\Desktop\School\For thesis\spv_images\InfraredSolarModules\module_metadata.json"
avg_glcmfeatures_nopreprocess_csv = r"C:\Users\mdrsp\Desktop\School\For thesis\csv_files\AVG_glcm_features_no_preprocessing.csv"

sample_img = r"C:\Users\mdrsp\Desktop\School\For thesis\spv_images\InfraredSolarModules\images\0.jpg" # path to sample image for testing the code

path_2_images = r"C:\Users\mdrsp\Desktop\School\For thesis\spv_images\InfraredSolarModules"

In [ ]:
# This script reads the data from the CSV file

import os
import csv
import json
import pandas as pd
import matplotlib.pylab as plt

# Set current directory to the folder
os.chdir("C:/Users/mdrsp/Downloads/InfraredSolarModules-master/InfraredSolarModules-master/2020-02-14_InfraredSolarModules/InfraredSolarModules")

x = []
y = []

with open('AVG_glcm_features_no_preprocessing.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader, None)
    wrote
        anomaly = row[1]
        homogeneity = float(row[2])
        contrast = float(row[3])
        energy = float(row[4])
        dissimilarity = float(row[5])
        asm = float(row[6])
        
        # appends x and y for svm 
        # x.append([homogeneity, contrast, energy, dissimilarity, asm])
        # y.append(anomaly)       

In [ ]:
# This script reads the data from the CSV file
# Set current directory to the folder

x = []
y = []

with open(avg_glcmfeatures_nopreprocess_csv, "r") as file:
    reader = csv.reader(file)
    next(reader, None)
    for row in reader:
        anomaly = row[1]
        homogeneity = float(row[2])
        contrast = float(row[3])
        energy = float(row[4])
        dissimilarity = float(row[5])
        asm = float(row[6])
        print(anomaly)     

In [ ]:
# Pre processing.
# function includes grascale conversion, Histogram equalization, adn noise filering
# gabor_filtering is used for noise filtering. 

def pre_processing(file):
    img = cv.imread(file)
    gray_img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    hist_img = cv.equalizeHist(gray_img)
    gabor_filter = create_gaborfilter()
    filtered_image = filtering(hist_img, gabor_filter) 
    type(filtered_image)
    return filtered_image

def create_gaborfilter():
    filters = []
    num = 16
    sigma = 1.0
    lambd = 30
    gamma = 0.25
    psi = 0
    for theta in np.arange(0, np.pi, np.pi/num):
        kern = cv.getGaborKernel((3,3), sigma, theta, lambd, gamma, psi, ktype=cv.CV_64F)
        kern /= 1.0*kern.sum()
        filters.append(kern)
    return filters

def filtering(img, filters):  
    newimage = np.zeros_like(img)
    depth = -1
    for kernels in filters:
        image_filter = cv.filter2D(img, depth, kernels) 
        np.maximum(newimage, image_filter, newimage)
    return newimage
    

In [ ]:
# Feature extraraction functions

# this function will be used to calculate the glcm for an indidivual image. 
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from skimage.io import imread_collection

def glcm_features(path):
    # needs cv2, numpy as np, from skimage.feature import graycomatrix, graycoprops from skimage.io import imread_collection
    img = cv.imread(path)
    
    image_gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    
    # Calculate the GLCM
    distances = [1, 2, 3]  # You can change the distances here
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # You can change the angles here
    glcm = graycomatrix(image_gray, distances=distances, angles=angles, levels=256,
                        symmetric=True, normed=True)
    
    # Calculate the GLCM features
    homogeneity = graycoprops(glcm, 'homogeneity')
    contrast = graycoprops(glcm, 'contrast')
    energy = graycoprops(glcm, 'energy')
    dissimilarity = graycoprops(glcm, 'dissimilarity')
    asm = graycoprops(glcm, 'ASM')
    
    # this averages them
    #homogeneity = float(np.average(homogeneity))
    #contrast = float(np.average(contrast))
    #energy = float(np.average(energy))
    #homogeneity = float(np.average(homogeneity))
    #dissimilarity = float(np.average(dissimilarity))
    #asm = float(np.average(asm))
    
    
    features = {'homogeneity'   : homogeneity,
                'contrast'      : contrast,
                'energy'        : energy,
                'dissimilarity' : dissimilarity,
                'asm'           : asm }
    return features


# Local binary Pattern
def local_binary_pattern(path):
    from skimage.feature import local_binary_pattern
    import matplotlib.pyplot as plt
    import numpy as np
    # Load the image
    img = plt.imread(path)
    # Compute the LBP of the grayscale image
    lbp_img = local_binary_pattern(img, P=8, R=1)
    # Convert the LBP image to uint8 format
    lbp_img = (lbp_img * 255).astype(np.uint8)
    return lbp_img

In [ ]:
# To visualize the features

from matplotlib.pyplot import figure

# Set current directory to the folder
os.chdir("C:/Users/mdrsp/Downloads/InfraredSolarModules-master/InfraredSolarModules-master/2020-02-14_InfraredSolarModules/InfraredSolarModules")

data = pd.read_csv('AVG_glcm_features_no_preprocessing.csv')

none = data[data.anomaly == 'No-Anomaly']
cracking = data[data.anomaly == 'Cracking']
cell = data[data.anomaly == 'Cell']
cell_multi = data[data.anomaly == 'Cell-Multi']
hotspot = data[data.anomaly == 'Hot-Spot']
hotspot_multi = data[data.anomaly == 'Hot-Spot-Multi']
offline = data[data.anomaly == 'Offline-Module']
vegetation = data[data.anomaly == 'Vegetation']
diode = data[data.anomaly == 'Diode']
diode_multi = data[data.anomaly == 'Diode-Multi']
soiling = data[data.anomaly == 'Soiling']
shadowing = data[data.anomaly == 'Shadowing']




In [ ]:
# This script writes all the features into a csv file
import os
import csv
import json
# Set current directory to the folder that has your json metat data file
os.chdir("C:/Users/mdrsp/Downloads/InfraredSolarModules-master/InfraredSolarModules-master/2020-02-14_InfraredSolarModules/InfraredSolarModules")

file = open("module_metadata.json")
data = json.load(file)
output = list()
count = 0
for key,value in data.items():
    current_img = value['image_filepath']
    anomaly = value['anomaly_class']
    # features = glcm_features(current_img)
    # data = {'image_filepath':current_img, 'anomaly':anomaly, 'homogeneity':features['homogeneity'], 'contrast':features['contrast'], 'energy':features['energy'], 'dissimilarity':features['dissimilarity'], 'asm':features['asm']}
    output.append(data)
    
        

headers = ['image_filepath', 'anomaly', 'homogeneity', 'contrast', 'energy', 'dissimilarity', 'asm']
with open('avg_glcm_values.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()
    writer.writerows(output)


In [ ]:
# Loads all the data in the json file into a dictionary for training

file = open(json_file)
data = json.load(file)

""" output = {"No-Anomaly":[], "Cracking":[], "Cell":[], "Cell-Multi":[], "Hot-Spot":[],
          "Hot-Spot-Multi":[], "Offline-Module":[], "Vegetation" : [], "Diode" : [], "Diode-Multi" : [],
          "Soiling":[], "Shadowing":[]} """

output = {"No-Anomaly":[], "Cracking":[], "Hot-Spot":[],  "Offline-Module":[], "Diode" : [], "Diode-Multi" : [], "Shadowing":[]}

for key,value in data.items():
    current_img = value['image_filepath']
    anomaly = value['anomaly_class']

    if anomaly == "Cell" or anomaly ==  "Cell-Multi" or anomaly == "Hot-Spot-Multi":
        anomaly = "Hot-Spot"
    elif anomaly =="Diode-Multi":
        anomaly = "Diode"
    elif anomaly == "Vegetation" or anomaly == "Soiling":
        anomaly = "Shadowing"
    
    output[anomaly].append(current_img)


In [ ]:
os.chdir(path_2_images)
images = []
labels = []

for anomaly,path in output.items():
    for img in path:
        lbp = local_binary_pattern(img)
        glcm = glcm_features(img)
        
        homogenity = np.ravel(glcm['homogeneity'])
        contrast = np.ravel(glcm['contrast'])
        energy = np.ravel(glcm['energy'])
        dissimilarity = np.ravel(glcm['dissimilarity'])
        asm = np.ravel(glcm['asm'])
        lbp = np.ravel(lbp)
        
        feature_vector = np.concatenate((homogenity, contrast, energy, dissimilarity, asm, lbp))
        feature_vector = np.ravel(feature_vector)
       
        images.append(feature_vector)
        labels.append(anomaly)

In [ ]:
# Convert the lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=50)

# Train an SVM classifier
clf = SVC(kernel='poly')
clf.fit(X_train, y_train)

# Test the SVM classifier
accuracy = clf.score(X_test, y_test)
print(f'Accuracy: {accuracy:.3f}')


In [ ]:
prediction = clf.predict(X_test)
print(classification_report(prediction, y_test))